# Programming Assignment

**Warning**:

Please modify only the code that is between
```Python
### YOUR CODE HERE ###
```
and
```Python
### ^^^^^^^^^^^^^^ ###
```
Some of the cells to be completed by the student are followed by a "check" cell that performs some basic unit tests. Run them after completing the code to check your results. Their only purpose is to help the student and will not be used to evaluate the assignment.

# Building a Bag of Words Model

Let's build a basic BoW model for a text classification problem with the [20 newsgroups](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) dataset.

In [ ]:
from sklearn.datasets import fetch_20newsgroups

X, y = fetch_20newsgroups(return_X_y=True, subset="all")

Using scikit-learn, create a pipeline consisting of a TF-IDF vectorization with 1-, 2- and 3-grams, and a Random Forest with 100 trees of max depth of 4. Evaluate it's f1-macro score with a 3-fold cross-validation:

In [ ]:
### YOUR CODE HERE ###
...
### ^^^^^^^^^^^^^^ ###

# Building a Recurrent Neural Network

<a name='0'></a>
## Notation

- Subscript $n$ denotes the $n^{th}$ sample. 

- Superscript $[l]$ denotes the $l^{th}$ layer. 

- Superscript $\langle t \rangle$ denotes the $t^{th}$ time step. 
    
- Subscript $(d)$ denotes the $d^{th}$ entry of a vector.

**Example**:

$z^{[2]<3>}_{1(4)}$ denotes the activation of the 1st training sample, [2]nd layer, <3>rd time step, and (4)th entry in the vector.

<a name='1'></a>
## 1 - Recurrent Neural Network (RNN)

### Input $x$

* A training sample $x_n$ consists of multiple time steps $T_{x_n}$. Let's assume $T_x$ be the number of timesteps in the longest $x_n$.
* A time step of an input sample, $x^{\langle t \rangle }_n$, is a one-dimensional input vector of size $n_x$. For example, a 1000-word vocabulary/dictionary would be encoded as a collection of one-hot vectors of size $n_x=D$ of 1000, and word $x_n^{\langle t \rangle}$ would have shape (1000,).
* Let's use mini-batches of size $m$ of $x_n$ samples to benefit from vectorization. The shape of one such batch will be $(n_x,m,T_x)$. For example, if mini-batches of size 10 are used, the resulting 3D tensor will be (5000,20,10)-shaped.
* For each time step $t$, a 2D slice $x^{\langle t \rangle}$ of shape $(n_x,m)$ will be fed into the network.

### Hidden state $z$

* The shape of a mini-batch of hidden states is $(n_z,m,T_x)$ including the time step dimension, with $n_z$ the number of hidden units.
* $z^{\langle t \rangle}$ is a 2D slice at timestep $t$ of this minibatch, and has a shape of $(n_z, m)$.

### Output $\hat{y}$
* $\hat{y}$ is a 3D tensor of shape $(n_y,m,T_y)$, with $n_{y}$ the number of units in the vector representing the prediction and $T_{y}$ the number of time steps in the prediction. Lets assume $T_y = T_x$.
* $\hat{y}^{\langle t \rangle}$ is a 2D slice for time step $t$ and has a shape of $(n_{y}, m)$.

<a name='1-1'></a>
### 1.1 - RNN Cell

Let's start by implementing the computations for a single time step of the RNN cell.

**Instructions**:
1. Compute the hidden state: $z^{\langle t \rangle} = \tanh(W_z z^{\langle t-1 \rangle} + W_x x^{\langle t \rangle} + b)$. Use [numpy.tanh](https://numpy.org/devdocs/reference/generated/numpy.tanh.html) and [numpy.dot](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html).
2. Use hidden state $z^{\langle t \rangle}$ to compute the prediction $\hat{y}^{\langle t \rangle} = softmax(W_y z^{\langle t \rangle} + b_y)$. Use the provided `softmax` function.
4. Return $z^{\langle t \rangle}$ , $\hat{y}^{\langle t \rangle}$

In [ ]:
import numpy as np

def softmax(x):
  expon = np.exp(x)
  return expon/np.sum(expon)

def rnn_cell(x, z, Wx, Wy, Wz, b, by):
    """
    Single step of the RNN cell.

    Arguments:
        x: input data at timestep "t", numpy array of shape (n_x,m).
        z: hidden state at timestep "t-1", numpy array of shape (n_z,m)
        Wx: weight matrix multiplying the input, numpy array of shape (n_z,n_x)
        Wy: weight matrix relating the hidden-state to the output, numpy array of shape (n_y,n_z)
        Wz: weight matrix multiplying the hidden state, numpy array of shape (n_z,n_z)
        b: bias, numpy array of shape (n_z,1)
        by: bias relating the hidden-state to the output, numpy array of shape (n_y,1)
    Returns:
        z: hidden state at timestep "t", of shape (n_z,m)
        y_pred: prediction at timestep "t", numpy array of shape (n_y,m)
    """
    ### YOUR CODE HERE ###
    z = ...
    y_pred = ...
    ### ^^^^^^^^^^^^^^ ###
    return z, y_pred

In [ ]:
# Check

np.random.seed(1)
x = np.random.randn(3, 10)
z = np.random.randn(5, 10)
Wx = np.random.randn(5, 3)
Wz = np.random.randn(5, 5)
Wy = np.random.randn(2, 5)
b = np.random.randn(5, 1)
by = np.random.randn(2, 1)
z, y_pred = rnn_cell(x, z, Wx, Wy, Wz, b, by)

np.testing.assert_equal(z.shape, (5, 10))
expected_z4 = np.array([-0.97683053, 0.99993218, 0.9999085, 0.99720415, 0.96532437, 0.24615069, 0.89433073, 0.88371261, 0.40821569, -0.99446565])
np.testing.assert_allclose(z[4], expected_z4, rtol=1e-2)
np.testing.assert_equal(y_pred.shape, (2, 10))
expected_y_pred1 = np.array([0.00402888, 0.03824577, 0.03569951, 0.07060504, 0.01322519, 0.03128045, 0.02207267, 0.25945201, 0.03835635, 0.00209625])
np.testing.assert_allclose(y_pred[1], expected_y_pred1, rtol=1e-2)

<a name='1-2'></a>
### 1.2 - RNN Forward Propagation 

A recurrent neural network can now be implemented as the repeated use of a cell. If the input sequence is $T_x$ time steps long, the RNN cell will be called $T_x$ times.

- The cell takes two inputs at each time step:
    - $z^{\langle t-1 \rangle}$: the hidden state from the previous time step
    - $x^{\langle t \rangle}$: the current time step's input data
- The cell produces two outputs at each time step:
    - $z^{\langle t \rangle}$: the hidden state at this time step
    - $\hat{y}^{\langle t \rangle}$: the prediction for this time step
- The weights and biases $(W_z, W_x, b)$ are re-used each time step 

**Instructions**:

At each time step $t$:
1. Get $x^{\langle t \rangle}$, the 2D slice of $x$ at time step $t$.
2. Update the 2D hidden state $z^{\langle t \rangle}$ and the prediction $\hat{y}^{\langle t \rangle}$ using `rnn_cell`.
3. Store the 2D hidden state in the 3D tensor $z$, at the $t^{th}$ position.
4. Store the 2D $\hat{y}^{\langle t \rangle}$ prediction in the 3D tensor $\hat{y}$ at the $t^{th}$ position.
5. Return the 3D tensors $z$ and $\hat{y}$.

In [ ]:
def rnn_forward(x, z, Wx, Wy, Wz, b, by):
    """
    Forward propagation of the RNN.

    Arguments:
        x: input data for every time-step, of shape (n_x,m,T_x).
        z: initial hidden state, of shape (n_z,m)
        Wx: weight matrix multiplying the input, numpy array of shape (n_z,n_x)
        Wy: weight matrix relating the hidden-state to the output, numpy array of shape (n_y,n_z)
        Wz: weight matrix multiplying the hidden state, numpy array of shape (n_z,n_z)
        b: bias numpy array of shape (n_z,1)
        by: bias relating the hidden-state to the output, numpy array of shape (n_y,1)
    Returns:
        z: hidden states for every time-step, numpy array of shape (n_z,m,T_x)
        y_pred: predictions for every time-step, numpy array of shape (n_y,m,T_x)
    """
    
    # Retrieve dimensions and initialize z, y_pred and zt
    n_x, m, T_x = x.shape
    n_y, n_z = Wy.shape
    zt = z
    z = np.zeros((n_z, m, T_x))
    y_pred = np.zeros((n_y, m, T_x))
    
    # loop over time steps
    for t in range(T_x):
        ### YOUR CODE HERE ###
        # Update the hidden state and compute the prediction with rnn_cell
        zt, yt_pred = ...
        # Save the value of the hidden state at time step t in z
        z[:,:,t] = ...
        # Save the value of the prediction at time step t in y_pred
        y_pred[:,:,t] = ...
        ### ^^^^^^^^^^^^^^ ###
    
    return z, y_pred

In [ ]:
# Check

np.random.seed(1)
x = np.random.randn(3, 10, 4)
z = np.random.randn(5, 10)
Wz = np.random.randn(5, 5)
Wx = np.random.randn(5, 3)
Wy = np.random.randn(2, 5)
b = np.random.randn(5, 1)
by = np.random.randn(2, 1)
z, y_pred = rnn_forward(x, z, Wx, Wy, Wz, b, by)

np.testing.assert_equal(z.shape, (5, 10, 4))
expected_z4 = np.array([-0.99999375, 0.77911235, -0.99861469, -0.99833267])
np.testing.assert_allclose(z[4][1], expected_z4, rtol=1e-2)
np.testing.assert_equal(y_pred.shape, (2, 10, 4))
expected_y_pred1 = np.array([0.01644489, 0.0328586, 0.0014877, 0.02851197])
np.testing.assert_allclose(y_pred[1][3], expected_y_pred1, rtol=1e-2)

<a name='2'></a>
## 2 - Long Short-Term Memory (LSTM)

The RNN works best when each output $\hat{y}^{\langle t \rangle}$ can be estimated using "local" context, but suffers from vanishing gradient and cannot remember information for many time steps. LSTM can cope with vanishing gradient much better.

Let's begin by implementing the LSTM cell for a single time step. Then, it will be possible to call it from a loop to have it process an input with $T_x$ time steps.

### Gates and states

#### Forget gate $f$

The forget gate is a tensor containing values between 0 and 1.
    * If a unit in the forget gate has a value close to 0, the LSTM will "forget" the stored previous state in the corresponding unit.
    * If a unit in the forget gate has a value close to 1, the LSTM will "remember" the the stored previous state in the corresponding unit.

$$f^{\langle t \rangle} = \sigma(\mathbf{W}_f[\mathbf{h}^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_f)$$

#### Candidate cell value $\tilde{\mathbf{c}}^{\langle t \rangle}$
* The candidate cell value is a tensor containing information from the current time step that may be stored in the current cell state $\mathbf{c}^{\langle t \rangle}$.
* The parts of the candidate cell value that get passed on depend on the input gate.
* The candidate cell value is a tensor containing values that range from -1 to 1.

$$\mathbf{\tilde{c}}^{\langle t \rangle} = \tanh\left( \mathbf{W}_{c} [\mathbf{h}^{\langle t - 1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_{c} \right)$$

#### Input gate $i$

* The input gate decides what entries of the candidate cell value $\tilde{\mathbf{c}}^{\langle t \rangle}$ to add to the cell state $c^{\langle t \rangle}$.
* The update gate is a tensor containing values between 0 and 1.
    * When a unit in the update gate is close to 1, it allows the value of the candidate $\tilde{\mathbf{c}}^{\langle t \rangle}$ to be passed onto the hidden state $\mathbf{c}^{\langle t \rangle}$
    * When a unit in the update gate is close to 0, it prevents the corresponding value in the candidate from being passed onto the hidden state.

$$i^{\langle t \rangle} = \sigma(\mathbf{W}_i[h^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_i)\tag{2}$$

#### Cell state $\mathbf{c}^{\langle t \rangle}$

* The cell state is the "memory" passed to future time steps.
* The new cell state $\mathbf{c}^{\langle t \rangle}$ is a combination of the previous cell state and the candidate value.

$$\mathbf{c}^{\langle t \rangle} = f^{\langle t \rangle}*\mathbf{c}^{\langle t-1 \rangle} + i^{\langle t \rangle}*\mathbf{\tilde{c}}^{\langle t \rangle}$$

#### Output gate $o$

* The output gate decides what gets sent as the output (prediction) of the time step.
* The output gate contains values that range from 0 to 1.

$$o^{\langle t \rangle}= \sigma(\mathbf{W}_o[\mathbf{h}^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_{o})$$ 


#### Hidden state $\mathbf{h}^{\langle t \rangle}$

* The hidden state gets passed to the LSTM cell's next time step.
* It is used to determine the three gates ($f, i, o$) of the next time step.
* The hidden state is also used for the prediction $y^{\langle t \rangle}$.

$$ \mathbf{h}^{\langle t \rangle} = o^{\langle t \rangle}*\tanh(\mathbf{c}^{\langle t \rangle})$$


#### Prediction $\mathbf{y}^{\langle t \rangle}_{pred}$

* The output or prediction.

If the problem is multiclass classification (therefore with a softmax activation at the output), for example, the equation would be:
$$\mathbf{y}^{\langle t \rangle}_{pred} = \textrm{softmax}(\mathbf{W}_{y} \mathbf{h}^{\langle t \rangle} + \mathbf{b}_{y})$$

<a name='2-1'></a>
### 2.1 - LSTM Cell

Lets implement the LSTM cell.

**Instructions**:
1. Compute all formulas for the gates and states. Use [numpy.tanh](https://docs.scipy.org/doc/numpy/reference/generated/numpy.tanh.html) and [numpy.dot](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html) and the provided functions `sigmoid` and `softmax`.
2. Compute the prediction $y^{\langle t \rangle}$.

In [ ]:
def sigmoid(x):
  return 1/(1 + np.exp(-x))

def lstm_cell(x, h, c, Wf, bf, Wi, bi, Wc, bc, Wo, bo, Wy, by):
    """
    Single step of the LSTM cell.

    Arguments:
        x: your input data at timestep "t", numpy array of shape (n_x,m).
        h: hidden state at timestep "t-1", numpy array of shape (n_h,m).
        c: memory state at timestep "t-1", numpy array of shape (n_h,m).
        Wf: weight matrix of the forget gate, numpy array of shape (n_h,n_h + n_x).
        bf: bias of the forget gate, numpy array of shape (n_h,1).
        Wi: weight matrix of the input gate, numpy array of shape (n_h,n_h + n_x).
        bi: bias of the input gate, numpy array of shape (n_h,1).
        Wc: weight matrix of the first "tanh", numpy array of shape (n_h,n_h + n_x).
        bc: bias of the first "tanh", numpy array of shape (n_h,1).
        Wo: weight matrix of the output gate, numpy array of shape (n_h,n_h + n_x).
        bo: bias of the output gate, numpy array of shape (n_h,1).
        Wy: weight matrix relating the hidden state to the output, numpy array of shape (n_y,n_h).
        by: bias relating the hidden state to the output, numpy array of shape (n_y,1).                    
    Returns:
        h_next: next hidden state, of shape (n_h,m).
        c_next: next memory state, of shape (n_h,m).
        yt_pred: prediction at timestep "t", numpy array of shape (n_y,m).
    """

    # Retrieve dimensions from shapes of x and Wy and concatenate h and x
    n_x, m = x.shape
    n_y, n_h = Wy.shape
    concat = np.concatenate((h, x))

    ### YOUR CODE HERE ###
    # Compute values for f, i, cct, c, o, h using the formulas
    f = ...
    i = ...
    cct = ...
    c = ...
    o = ...
    h = ...

    # Compute prediction of the LSTM cell
    y_pred = ...
    ### ^^^^^^^^^^^^^^ ###

    return h, c, y_pred

In [ ]:
# Check

np.random.seed(1)
x = np.random.randn(3, 10)
h = np.random.randn(5, 10)
c = np.random.randn(5, 10)
Wf = np.random.randn(5, 5 + 3)
bf = np.random.randn(5, 1)
Wi = np.random.randn(5, 5 + 3)
bi = np.random.randn(5, 1)
Wo = np.random.randn(5, 5 + 3)
bo = np.random.randn(5, 1)
Wc = np.random.randn(5, 5 + 3)
bc = np.random.randn(5, 1)
Wy = np.random.randn(2, 5)
by = np.random.randn(2, 1)
h, c, y_pred = lstm_cell(x, h, c, Wf, bf, Wi, bi, Wc, bc, Wo, bo, Wy, by)

np.testing.assert_equal(h.shape, (5, 10))
expected_h4 = np.array([-0.66408471, 0.0036921, 0.02088357, 0.22834167, -0.85575339, 0.00138482, 0.76566531, 0.34631421, -0.00215674, 0.43827275])
np.testing.assert_allclose(h[4], expected_h4, rtol=1e-2)
np.testing.assert_equal(c.shape, (5, 10))
expected_c2 = np.array([0.63267805, 1.00570849, 0.35504474, 0.20690913, -1.64566718, 0.11832942, 0.76449811, -0.0981561, -0.74348425, -0.26810932])
np.testing.assert_allclose(c[2], expected_c2, rtol=1e-2)
np.testing.assert_equal(y_pred.shape, (2, 10))
expected_y_pred1 = np.array([0.0283805, 0.00706782, 0.00753306, 0.01091349, 0.02806865, 0.00955958, 0.00444923, 0.01152125, 0.01426974, 0.01237766])
np.testing.assert_allclose(y_pred[1], expected_y_pred1, rtol=1e-2)

<a name='2-2'></a>
### 2.2 - LSTM Forward Propagation

Now it is possible to iterate over the LSTM cell using a for loop to process a sequence of $T_x$ inputs.

**Instructions**:

For each time step:
1. From the 3D tensor $x$, get a 2D slice $x^{\langle t \rangle}$ at time step $t$.
2. Call the `lstm_cell` function to get the hidden state, cell state and prediction.
3. Store the hidden state, cell state and prediction (the 2D tensors) inside the 3D tensors.

In [ ]:
def lstm_forward(x, h, Wf, bf, Wi, bi, Wc, bc, Wo, bo, Wy, by):
    """
    Forward propagation of the LSTM network.

    Arguments:
        x: input data for every time step, of shape (n_x,m,T_x).
        h: initial hidden state, of shape (n_h,m).
        Wf: weight matrix of the forget gate, numpy array of shape (n_h,n_h + n_x).
        bf: bias of the forget gate, numpy array of shape (n_h,1).
        Wi: weight matrix of the input gate, numpy array of shape (n_h,n_h + n_x).
        bi: bias of the input gate, numpy array of shape (n_h,1).
        Wc: weight matrix of the first "tanh", numpy array of shape (n_h,n_h + n_x).
        bc: bias of the first "tanh", numpy array of shape (n_h,1).
        Wo: weight matrix of the output gate, numpy array of shape (n_h,n_h + n_x).
        bo: bias of the output gate, numpy array of shape (n_h,1).
        Wy: weight matrix relating the hidden state to the output, numpy array of shape (n_y,n_h).
        by: bias relating the hidden state to the output, numpy array of shape (n_y,1).
    Returns:
        a: hidden states for every time step, numpy array of shape (n_h,m,T_x).
        y: predictions for every time step, numpy array of shape (n_y,m,T_x).
        c: cell state, numpy array of shape (n_h,m,T_x).
    """

    # Retrieve dimensions from shapes of x and Wy and initialize h, c and y
    n_x, m, T_x = x.shape
    n_y, n_h = Wy.shape
    ht = h
    ct = np.zeros((n_h, m))
    h = np.zeros((n_h, m, T_x))
    c = np.zeros((n_h, m, T_x))
    y_pred = np.zeros((n_y, m, T_x))
    
    # loop over time steps
    for t in range(T_x):
        ### YOUR CODE HERE ###
        # Get the 2D slice 'xt' from the 3D input 'x' at time step 't'
        xt = ...
        # Update next hidden state and next memory state and compute the prediction with lstm_cell
        ht, ct, yt_pred = ...
        # Save the value of the new "next" hidden state in h
        h[:,:,t] = ...
        # Save the value of the next cell state in c
        c[:,:,t]  = ...
        # Save the value of the prediction in y_pred
        y_pred[:,:,t] = ...
        ### ^^^^^^^^^^^^^^ ###
    
    return h, c, y_pred

In [ ]:
# Check

np.random.seed(1)
x = np.random.randn(3, 10, 7)
h = np.random.randn(5, 10)
Wf = np.random.randn(5, 5 + 3)
bf = np.random.randn(5, 1)
Wi = np.random.randn(5, 5 + 3)
bi= np.random.randn(5, 1)
Wo = np.random.randn(5, 5 + 3)
bo = np.random.randn(5, 1)
Wc = np.random.randn(5, 5 + 3)
bc = np.random.randn(5, 1)
Wy = np.random.randn(2, 5)
by = np.random.randn(2, 1)
h, c, y_pred = lstm_forward(x, h, Wf, bf, Wi, bi, Wc, bc, Wo, bo, Wy, by)

np.testing.assert_equal(h.shape, (5, 10, 7))
expected_h436 = 0.17211776753291672
np.testing.assert_allclose(h[4][3][6], expected_h436, rtol=1e-2)
np.testing.assert_equal(c.shape, (5, 10, 7))
expected_c121 = -0.8555449167181981
np.testing.assert_allclose(c[1][2][1], expected_c121, rtol=1e-2)
np.testing.assert_equal(y_pred.shape, (2, 10, 7))
expected_y_pred143 = 0.10837052997887556
np.testing.assert_allclose(y_pred[1][4][3], expected_y_pred143, rtol=1e-2)

# Building a Transformer Neural Network

Finally, let's build a new classifier, this time for the IMDB dataset, using a Transformer Neural Network.

In [ ]:
import keras

vocab_size = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=vocab_size)
X_train = keras.utils.pad_sequences(X_train, maxlen=maxlen)
X_test = keras.utils.pad_sequences(X_test, maxlen=maxlen)

Using keras, implement a transformer block and a token and position embedding as layers, and use them to build a classifier. Train it for 1 epoch with Adam on the training partition while using the test partition to calculate the validation loss and accuracy at every epoch:

In [ ]:
### YOUR CODE HERE ###
...
### ^^^^^^^^^^^^^^ ###